# 函数装饰器和闭包

# 装饰器的基础知识

严格来说，装饰器只是语法糖。方便元编程（在运行时改变程序的行为）

两大作用：
- 把被修饰函数替换成其他函数
- 在加载模块时立即执行

In [3]:
# 一个简单的例子
# 使用装饰器将返回的函数替换
# 装饰器就是一个以被修饰函数为参数的函数

def deco(func):
    # 在装饰器函数内创建另一个函数并返回
    def inner():
        print("inner() is running!")
    return inner

@deco
def target():
    print("target() is running")

# 执行被修饰函数
target()

inner() is running!


In [5]:
# 将函数作为对象

target  # target成为函数inner的引用

<function __main__.deco.<locals>.inner()>

# Python何时执行装饰器

装饰器在导入模块时就会执行。

python中提到的导入时和运行时，前者指的就是import的时候就会运行的代码。

In [1]:
import registration

running register (<function f1 at 0x7fdc99060310>)
running register (<function f2 at 0x7fdc9aeb55e0>)


### 装饰器函数在定义函数之后立即执行。因此在导入时也会即刻执行，即使我们没有调用任何模块内的函数。


## 实际应用时两个特殊的地方：
### 1 装饰器函数和被装饰的函数在同一个模块中定义，但现实中经常是装饰器函数在一个模块中定义，然后应用的其他模块的函数上。
### 2 示例中装饰器函数返回了输入的函数，但现实中往往会在装饰器中定义一个新的函数并返回。

# 使用装饰器改进“策略”模式

In [6]:
from collections import namedtuple

Customer = namedtuple('Customer','name fidelity')

class LineItem:
    '''每种产品信息，产品名，数量，单价等'''
    def __init__(self, product, quantity, price):
        self.product = product
        self.quantity = quantity
        self.price = price

    def total(self):
        return self.price * self.quantity


class Order:
    '''账单'''
    def __init__(self, customer, cart, promotion=None):
        self.customer = customer
        self.cart = list(cart)
        self.promotion = promotion

    def total(self):
        if not hasattr(self, '__total'):
            self.__total = sum(item.total() for item in self.cart)
        return self.__total

    def due(self):
        if self.promotion is None:
            discount = 0.0
        else:
            discount = self.promotion(order = self) # 此处promotion变量已经是一个函数了，直接执行括号运算符就行了。
        return self.total() - discount

    def __repr__(self):
        fm = "<Order total:{:.2f} due:{:.2f}>"
        return fm.format(self.total(),self.due())

# 全局变量，用来存储所有策略函数
promo = []

# 创建一个修饰器函数，将所有策略函数注册到全局变量中。
def promotion(func):
    promo.append(func)
    return func

@promotion
def fidelity_promo(order):
    '''满1000积分提供5%的折扣'''
    return order.total() * 0.05 if order.customer.fidelity >= 1000.0 else 0.0

@promotion
def bulkitem_promo(order):
    '''单个商品数量为20或以上时提供10%的折扣'''
    d = 0.0
    for item in order.cart:
        if item.quantity >= 20 :
            d += item.total() * 0.1
    return d

@promotion
def largeorder_promo(order):
    '''订单中达到或超过10种商品时提供7%的折扣'''
    distinct_item = { item.product for item in order.cart } #使用了集，集内不能有重复元素
    if len(distinct_item) >= 10 :
        return order.total() * 0.07
    return 0.0

def best_promo(order):
    '''返回最大优惠方案'''
    return max([pr(order) for pr in promo ])


In [9]:
joe = Customer('John Doe', 0)
ann = Customer('Ann Smith', 1100)
cart = [LineItem('banana', 4, 0.5),
        LineItem('apple', 15,1.5),
        LineItem('watermelon', 30, 5.0)]


Order(ann, cart, best_promo)

<Order total:174.50 due:159.50>

# 变量作用域规则

In [17]:
# 一个正常的例子 我们故意不去定义全局变量b
def f1(a):
    print(a)
    print(b)

f1(3)

3
9


In [18]:
# 一个不太好理解的例子：
b = 6
def f2(a):
    print(a)
    print(b)
    b = 9

f2(3)

3


UnboundLocalError: local variable 'b' referenced before assignment

上面例子返回的错误提示是局部变量b在定义前就被引用了。而忽视了我们之前定义的全局变量b。

这是python的一个语言设计：

**python不要求声明变量，但是假定在函数定义体中赋值的变量是局部变量。**

In [20]:
# 使用global关键字来声明变量即可：
b = 6
def f3(a):
    global b
    print(a)
    print(b)
    b = 9

f3(3)


3
6


In [21]:
b

9

In [22]:
from dis import dis
dis(f2)

  4           0 LOAD_GLOBAL              0 (print)
              2 LOAD_FAST                0 (a)
              4 CALL_FUNCTION            1
              6 POP_TOP

  5           8 LOAD_GLOBAL              0 (print)
             10 LOAD_FAST                1 (b)
             12 CALL_FUNCTION            1
             14 POP_TOP

  6          16 LOAD_CONST               1 (9)
             18 STORE_FAST               1 (b)
             20 LOAD_CONST               0 (None)
             22 RETURN_VALUE


In [23]:
dis(f3)

  5           0 LOAD_GLOBAL              0 (print)
              2 LOAD_FAST                0 (a)
              4 CALL_FUNCTION            1
              6 POP_TOP

  6           8 LOAD_GLOBAL              0 (print)
             10 LOAD_GLOBAL              1 (b)
             12 CALL_FUNCTION            1
             14 POP_TOP

  7          16 LOAD_CONST               1 (9)
             18 STORE_GLOBAL             1 (b)
             20 LOAD_CONST               0 (None)
             22 RETURN_VALUE


# 闭包

### 闭包指延伸了作用域的函数，其中包含函数定义体中引用，但不在定义体中定义的非全局变量。

In [28]:
# 使用类实现：可以记录之前的值，累积后再求平均输出结果。
class make_averager_c():
    def __init__(self):
        self._series = []

    def __call__(self, value):
        self._series.append(value)
        total = sum(self._series)
        return total/len(self._series)

In [29]:
avg_c = make_averager_c()
avg_c(10)

10.0

In [30]:
avg_c(15)

12.5

In [31]:
avg_c(20)

15.0

In [24]:
# 使用函数实现

def make_averager():
    series = []

    def averager(new_value):
        series.append(new_value)
        total = sum(series)
        return total/len(series)
    
    return averager

In [25]:
avg = make_averager()
avg(10)

10.0

In [26]:
avg(15)

12.5

In [27]:
avg(20)

15.0

In [32]:
# 两条曲线之间的范围即为闭包：变量series相对于函数averager()来说就是在函数作用域之外了，但是依然能够引用到。
# 此时，series 为自由变量，而非局部变量

def make_averager():
    #------------------------------
    series = []

    def averager(new_value):
        series.append(new_value)
        total = sum(series)
        return total/len(series)
    #-----------------------------
    return averager

In [41]:
avg = make_averager()
avg(10)

10.0

In [42]:
avg(15)

12.5

In [43]:
avg(20)

15.0

In [44]:
avg.__code__.co_varnames #局部变量

('new_value', 'total')

In [45]:
avg.__code__.co_freevars #自由变量

('series',)

In [46]:
avg.__closure__

(<cell at 0x7fdc99b71d60: list object at 0x7fdc9a72ed80>,)

In [47]:
avg.__closure__[0].cell_contents

[10, 15, 20]

### 闭包是一种函数，它会保留定义函数时存在的自由变量的绑定，这样调用函数时，虽然定义作用域不可用了，但是仍能使用那些绑定。

> 只有嵌套在其他函数内部的函数，才有成为闭包的可能。因为它有可能调用外部的非全局变量。

# nonlocal声明

In [52]:
# 继续优化上面的求平均函数：

def make_averager2():
    count = 0
    total = 0
    def averager2(new_value):
        total += new_value
        count += 1
        return total/count
    return averager2

In [53]:
a = make_averager2()

a(10)

UnboundLocalError: local variable 'total' referenced before assignment

这里之所以会出现错误，而且提示total成为了局部变量，而不是自由变量，那是因为**total是一个不可变类型。**

不可变类型（数字，字符串，元组）其实只能读，不能写。一旦写了，python会隐藏地创建了一个新的变量。

而上面的例子中，series是数组，是可变类型。也没有直接赋值给它，而是调用了append函数。因此上面的例子没有问题。

解决办法： 使用**nonlocal**关键字来声明一下自由变量。


In [54]:
# 使用nonlocal来声明不可变变量为自由变量！

def make_averager3():
    count = 0
    total = 0
    def averager3(new_value):
        nonlocal count,total # 在调用和赋值的函数中声明
        total += new_value
        count += 1
        return total/count
    return averager3

In [55]:
    c = make_averager3()
    c(10)

10.0

In [56]:
c(15)

12.5

In [57]:
c(20)

15.0

# 实现一个简单的装饰器

In [58]:
import time
from clockdeco import clock

@clock
def snooze(second):
    time.sleep(second)


snooze(2)

[2.00410020s], snooze(2)->None


In [60]:
@clock
def factorial(n):
    return 1 if n < 2 else n*factorial(n-1)


factorial(2)

[0.00000037s], factorial(1)->1
[0.00015563s], factorial(2)->2


2

In [63]:
# 装饰器将函数绑定给了装饰器内部函数：
factorial.__name__

'clocked'

### 这是装饰器的典型行为:把被装饰的函数替换成新函数，二者接受相同 的参数，而且(通常)返回被装饰的函数本该返回的值，同时还会做些 额外操作。

In [28]:
from clockdeco2 import clock

@clock
def factorial2(num):
    return 1 if num < 2 else num * factorial2(num-1)

@clock
def test(*args, **kwargs):
    log = ', '.join(["%s "%(value) for value in args])
    log += ', '.join(["%s : %s"%(name, value) for name , value in kwargs.items()])
    return log


test('dd', a='c')

TypeError: clocked() got an unexpected keyword argument 'a'

# 标准库中的修饰器


比较常用的修饰器：
- property ： 19.2节介绍
- classmethod ：9.4节介绍
- staticmethod ：9.4节介绍
- functools.wraps ： 协助构建行为良好的修饰器
- functools.lru_cache : 它把耗时的函数的结果保存起来，避免传入相同的参数时重复计算。LRU 三个字母是“Least Recently Used”的缩写。
- functools.singledispatch

## functools.lru_cache缓存递归函数的中间结果

因为 lru_cache 使用字典存储结果，而且键根据调用时传
入的定位参数和关键字参数创建，所以被 lru_cache 装饰的函数，它 的所有参数都必须是可散列的。

In [98]:
from clockdeco import clock

@clock
def fibonacci(n):
    if n < 2 :
        return n
    return fibonacci(n-1) + fibonacci(n-2)


fibonacci(8)

[0.00000058s], fibonacci(1)->1
[0.00000071s], fibonacci(0)->0
[0.00092309s], fibonacci(2)->1
[0.00000077s], fibonacci(1)->1
[0.00112688s], fibonacci(3)->2
[0.00000052s], fibonacci(1)->1
[0.00000083s], fibonacci(0)->0
[0.00070626s], fibonacci(2)->1
[0.00209366s], fibonacci(4)->3
[0.00000051s], fibonacci(1)->1
[0.00000062s], fibonacci(0)->0
[0.00020043s], fibonacci(2)->1
[0.00000197s], fibonacci(1)->1
[0.00179201s], fibonacci(3)->2
[0.00412892s], fibonacci(5)->5
[0.00000059s], fibonacci(1)->1
[0.00000072s], fibonacci(0)->0
[0.00020438s], fibonacci(2)->1
[0.00000071s], fibonacci(1)->1
[0.00041063s], fibonacci(3)->2
[0.00000050s], fibonacci(1)->1
[0.00000049s], fibonacci(0)->0
[0.00050632s], fibonacci(2)->1
[0.00109188s], fibonacci(4)->3
[0.00537671s], fibonacci(6)->8
[0.00000034s], fibonacci(1)->1
[0.00000039s], fibonacci(0)->0
[0.00011062s], fibonacci(2)->1
[0.00000099s], fibonacci(1)->1
[0.00056474s], fibonacci(3)->2
[0.00000058s], fibonacci(1)->1
[0.00000064s], fibonacci(0)->0
[0.00021

21

In [99]:
from clockdeco import clock
import functools

# lru_cache作用于clock修饰返回的函数。如果顺序调换，就不对了！！！
@functools.lru_cache
@clock
def fibonacci(n):
    if n < 2 :
        return n
    return fibonacci(n-1) + fibonacci(n-2)


fibonacci(30)

[0.00000042s], fibonacci(1)->1
[0.00000060s], fibonacci(0)->0
[0.00039008s], fibonacci(2)->1
[0.00045389s], fibonacci(3)->2
[0.00051440s], fibonacci(4)->3
[0.00057429s], fibonacci(5)->5
[0.00063515s], fibonacci(6)->8
[0.00069588s], fibonacci(7)->13
[0.00075633s], fibonacci(8)->21
[0.00081657s], fibonacci(9)->34
[0.00087752s], fibonacci(10)->55
[0.00093953s], fibonacci(11)->89
[0.00100034s], fibonacci(12)->144
[0.00106105s], fibonacci(13)->233
[0.00112137s], fibonacci(14)->377
[0.00118104s], fibonacci(15)->610
[0.00140605s], fibonacci(16)->987
[0.00151805s], fibonacci(17)->1597
[0.00160385s], fibonacci(18)->2584
[0.00167972s], fibonacci(19)->4181
[0.00174291s], fibonacci(20)->6765
[0.00180859s], fibonacci(21)->10946
[0.00187791s], fibonacci(22)->17711
[0.00196103s], fibonacci(23)->28657
[0.00207032s], fibonacci(24)->46368
[0.00217286s], fibonacci(25)->75025
[0.00230053s], fibonacci(26)->121393
[0.00261563s], fibonacci(27)->196418
[0.00278960s], fibonacci(28)->317811
[0.00299329s], fibon

832040

#  functools.singledispatch

python不支持函数/方法重载。

使用 @singledispatch 装饰的普通函数会变成泛函数(genericfunction):根据第一个参数的类型，以不同方式执行相同操作的一组函数

singledispatch 创建一个自定义的 htmlize.register 装饰器，把多个函数绑在一起组成一个泛函数


In [2]:
from functools import singledispatch
from collections import abc
import numbers
import html


@singledispatch
def htmlize(obj):
    content = html.escape(repr(obj))
    return '<pre>{}</pre>'.format(content)


@htmlize.register(str)
def _(text):
    content = html.escape(text).replace('\n', '<br>\n')
    return '<p>{0}</p>'.format(content)



In [8]:
htmlize.register.__code__

<code object register at 0x7fd45cb95190, file "/Users/liuzhongkai/opt/anaconda3/lib/python3.8/functools.py", line 837>

# 叠放装饰器

In [ ]:
@d1
@d2
def f():
    pass

# 等价于：

r = d1(d2(f))

# 参数化装饰器

In [33]:
# 将之前策略方案中的修饰器进行改写。添加一个额外的参数————是否开启。
registry = set()
def register(active = True):    # 修饰器函数不再用func作为参数导入，而此时该函数为修饰器工厂函数！！！
    def decorate(func):         # 此为真正的修饰器函数！其参数是一个函数！
        print("register(active = %s) -> decorate(%s)"%(active, func))
        def decorate_inner_func(*args): # 修饰器内部的函数，用来做一些操作！
            print("decorate_inner_func is running")
            if active:
                registry.add(func)
            else:
                registry.discard(func)
            return func
        return decorate_inner_func
    return decorate


@register(True)
def func01():
    print("func01 is running! \n")

@register() # 使用装饰器工厂函数时，需要在后面添加(),可以理解为： registter(active)(func),因为修饰器默认隐藏参数，因此第二个括号就可以不写了，但是第一个要保留下来！
def func02():
    print("func02 is running! \n")

@register(False)
def func03():
    print("func03 is running! \n")

@register(False)
def func02():
    print("func03 is running! \n")

func01()() # 如果想的到func的返回只，我们需要再次让函数运算，因此使用了两个括号。看起来有些奇怪。
# 我们可以将上面的decorate_inner_func（）的返回值改为return func()，这里就不用使用两个括号的奇怪写法。


[repr(f) for f in registry] # 没有func02和func03，因为func02第二次又给去除了，func03压根就没有注册进去


register(active = True) -> decorate(<function func01 at 0x7fd464f0dee0>)
register(active = True) -> decorate(<function func02 at 0x7fd462a14550>)
register(active = False) -> decorate(<function func03 at 0x7fd464f0d4c0>)
register(active = False) -> decorate(<function func02 at 0x7fd4655ddca0>)
decorate_inner_func is running
func01 is running! 



['<function func01 at 0x7fd464f0dee0>']

In [46]:
# 参数化clock修饰器
import time


DEFUALT_FMT = "[{elapsed:.8f}s] {name}({args}) -> {result}"
def clock_n(fmt = DEFUALT_FMT):
    def decorate(func):
        def clocked(*_args):
            t0 = time.time()
            _result = func(*_args)
            elapsed = time.time() - t0
            name = func.__name__
            args = ', '.join([repr(arg) for arg in _args])
            result = repr(_result)
            print(fmt.format(**locals()))
            return _result
        return clocked
    return decorate



FMT = "[{elapsed:.8f}s] {name}({args})"
@clock_n(FMT)
def factorial_new(num):
    if num < 2:
        return 1
    return factorial_new(num - 1) * num

factorial_new(4)


[0.00000000s] factorial_new(1)
[0.00036287s] factorial_new(2)
[0.00039220s] factorial_new(3)
[0.00041223s] factorial_new(4)


24

# 7.11 本章小结 
本章介绍了很多基础知识，虽然学习之路崎岖不平，我还是尽可能让路
途平坦顺畅。毕竟，我们已经进入元编程领域了。 

开始，我们先编写了一个没有内部函数的 @register 装饰器;最后，
我们实现了有两层嵌套函数的参数化装饰器 @clock()。 尽管注册装饰器在多数情况下都很简单，但是在高级的 Python 框架中却
有用武之地。我们使用注册方式对第 6 章的“策略”模式做了重构。

参数化装饰器基本上都涉及至少两层嵌套函数，如果想使用 @functools.wraps 生成装饰器，为高级技术提供更好的支持，嵌套 层级可能还会更深，比如前面简要介绍过的叠放装饰器。

我们还讨论了标准库中 functools 模块提供的两个出色的函数装饰 器:@lru_cache() 和 @singledispatch。

若想真正理解装饰器，需要区分导入时和运行时，还要知道变量作用 域、闭包和新增的 nonlocal 声明。掌握闭包和 nonlocal 不仅对构
建装饰器有帮助，还能协助你在构建 GUI 程序时面向事件编程，或者使 用回调处理异步 I/O。